In [55]:
import torch
import torch.nn as nn
import math

class RotaryEmbedding(nn.Module):
    def __init__(self, hidden_size, num_heads, base, max_len):
        """
        RoPE位置编码模块。
        Args:
            hidden_size (int): 模型维度
            num_heads (int): 注意力头数量
            base (int): 频率基值
            max_len (int): 最大序列长度
        """
        super(RotaryEmbedding, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.base = base
        self.max_len = max_len
        self.head_dim = hidden_size // num_heads
        self.cos_pos_cache, self.sin_pos_cache = self._compute_pos_emb()
    
    def _compute_pos_emb(self):
        """
        计算位置编码的余弦和正弦值。
        Returns:
            cos_pos (Tensor): 余弦位置编码
            sin_pos (Tensor): 正弦位置编码
        """
        theta_i = 1. / (self.base ** (torch.arange(0, self.head_dim, 2).float() / self.head_dim))
        positions = torch.arange(self.max_len)
        pos_emb = positions.unsqueeze(1) * theta_i.unsqueeze(0)
        
        cos_pos = pos_emb.sin().repeat_interleave(2, dim=-1)
        sin_pos = pos_emb.cos().repeat_interleave(2, dim=-1)
        
        return cos_pos, sin_pos
     
    def forward(self, q):
        """
        RoPE位置编码应用。
        Args:
            q (torch.Tensor): 输入张量 [bs, num_heads, seq_len, head_dim]
        Returns:
            torch.Tensor: 应用位置编码后的张量
        """
        bs, seq_len = q.shape[0], q.shape[2]
        cos_pos = self.cos_pos_cache[:seq_len].to(q.device) # [seq_len, head_dim]
        sin_pos = self.sin_pos_cache[:seq_len].to(q.device) # [seq_len, head_dim]
        
        # 扩展维度以匹配batch和head维度
        cos_pos = cos_pos.unsqueeze(0).unsqueeze(0) # [1, 1, seq_len, head_dim]
        sin_pos = sin_pos.unsqueeze(0).unsqueeze(0) # [1, 1, seq_len, head_dim]
        
        # RoPE变换
        q2 = torch.stack([-q[..., 1::2], q[..., ::2]], dim=-1) # 奇偶交替
        q2 = q2.reshape(q.shape).contiguous()
        
        return q * cos_pos + q2 * sin_pos
    

class MultiHeadLatentAttention(nn.Module):
    def __init__(self, hidden_size, down_dim, up_dim, num_heads, rope_head_dim, base, max_len, dropout, bias, device):      
        """
        Multi-Head Latent Attention 实现。
        Args:
            hidden_size (int): 输入特征维度。
            down_dim (int): 降维后的维度。
            up_dim (int): 升维后的维度。
            num_heads (int): 注意力头数量。
            rope_head_dim (int): RoPE编码的头维度。
            dropout (float): ddropout概率。
            bias (bool): 是否使用偏置。
            device (str): 设备类型（'cpu'或'cuda'）。
        """
        super(MultiHeadLatentAttention, self).__init__()
        
        self.hidden_size = hidden_size
        self.down_dim = down_dim
        self.up_dim = up_dim
        self.num_heads = num_heads
        self.rope_head_dim = rope_head_dim
        self.head_dim = hidden_size // num_heads
        self.v_head_dim = up_dim // num_heads
        
        # 降维投影
        self.down_proj_kv = nn.Linear(hidden_size, down_dim, bias, device)
        self.down_proj_q = nn.Linear(hidden_size, down_dim, bias, device)
        
        # 升维投影
        self.up_proj_k = nn.Linear(down_dim, up_dim, bias, device)
        self.up_proj_v = nn.Linear(down_dim, up_dim, bias, device)
        self.up_proj_q = nn.Linear(down_dim, up_dim, bias, device)
        
        # 解耦Q/K投影
        self.proj_qr = nn.Linear(down_dim, rope_head_dim * num_heads, bias, device)
        self.proj_kr = nn.Linear(hidden_size, rope_head_dim, bias, device)
        
        # RoPE位置编码
        self.rope_q = RotaryEmbedding(rope_head_dim * num_heads, num_heads, base, max_len)
        self.rope_k = RotaryEmbedding(rope_head_dim, 1, base, max_len)
        
        # 输出层
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_heads * self.v_head_dim, hidden_size, bias, device)
        self.res_dropout = nn.Dropout(dropout)
        
    def forward(self, hidden_state, mask=None):
        """
        前向传播。
        Args:
            h (torch.Tensor): 输入张量 [batch_size, seq_len, hidden_size]
            mask (torch.Tensor): 注意力掩码 [batch_size, seq_len]
        Returns:
            torch.Tensor: 输出张量 [bs, seq_len, d_model]
        """
        batch_size, seq_len, _ = hidden_state.size()
        
        # 1. 低秩转换
        c_t_kv = self.down_proj_kv(hidden_state) # [batch_size, seq_len, down_dim]
        k_t_c = self.up_proj_k(c_t_kv) # [batch_size, seq_len, up_dim]
        v_t_c = self.up_proj_v(c_t_kv) # [batch_size, seq_len, up_dim]
        c_t_q = self.down_proj_q(hidden_state) # [batch_size, seq_len, down_dim]
        q_t_c = self.up_proj_q(c_t_q) # [batch_size, seq_len, up_dim]
        
        # 2. 解耦Q/K处理
        # RoPE投影处理
        q_t_r = self.proj_qr(c_t_q) # [batch_size, seq_len, rope_head_dim * num_heads]
        q_t_r = q_t_r.view(batch_size, seq_len, self.num_heads, self.rope_head_dim).transpose(1, 2) # [batch_size, num_heads, seq_len, rope_head_dim]
        q_t_r = self.rope_q(q_t_r) # 应用RoPE编码
        
        k_t_r = self.proj_kr(hidden_state) # [batch_size, seq_len, rope_head_dim]
        k_t_r = k_t_r.unsqueeze(1) # [batch_size, 1, seq_len, rope_head_dim]
        k_t_r = self.rope_k(k_t_r) # 应用RoPE编码
        
        # 3. 注意力计算
        # Q/K/V维度调整
        q_t_c = q_t_c.view(batch_size, seq_len, self.num_heads, -1).transpose(1, 2) # [batch_size, num_heads, seq_len, v_head_dim]
        q = torch.cat([q_t_c, q_t_r], dim=-1) # [batch_size, num_heads, seq_len, (up_dim+rope_head_dim)/num_heads]
        
        k_t_c = k_t_c.view(batch_size, seq_len, self.num_heads, -1).transpose(1, 2) # [batch_size, num_heads, seq_len, v_head_dim]
        k_t_r = k_t_r.expand(batch_size, self.num_heads, seq_len, -1) # [batch_size, num_heads, seq_len, rope_head_dim]
        k = torch.cat([k_t_c, k_t_r], dim=-1) # [batch_size, num_heads, seq_len, (up_dim+rope_head_dim)/num_heads]
        
        # 4. 计算注意力权重
        scores = torch.matmul(q, k.transpose(-1, -2)) # [batch_size, num_heads, seq_len, seq_len]
        scores = scores / (math.sqrt(self.head_dim) + math.sqrt(self.rope_head_dim))
        
        # 5. attention mask
        if attention_mask is not None:
            scores = scores.masked_fill(mask[:, None, None, :] == 0, float('-inf')) # [batch_size, num_heads, seq_len, seq_len]
        
        attention_weights = torch.softmax(scores, dim=-1) # [batch_size, num_heads, seq_len, seq_len]
        attention_weights = self.dropout(attention_weights)
        
        # 6. V维度调整
        v_t_c = v_t_c.view(batch_size, seq_len, self.num_heads, self.v_head_dim).transpose(1, 2) # [batch_size, num_heads, seq_len, v_head_dim]
        
        # 7. 计算上下文向量
        context = torch.matmul(attention_weights, v_t_c) # [batch_size, num_heads, seq_len, v_head_dim]
        
        # 8. 合并多头
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, -1) # [batch_size, seq_len, num_heads * v_head_dim]
        
        # 9. 输出投影
        output = self.fc(context) # [batch_size, seq_len, hidden_size]
        output = self.res_dropout(output)
            
        return output

In [56]:
if __name__ == "__main__":
    # 示例
    batch_size = 2
    seq_len = 10
    hidden_size = 256
    num_heads = 8
    base=10000
    max_len=512
    down_dim=64
    up_dim=128
    rope_head_dim=26
    dropout = 0.1
    bias = False
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 创建一个 MLA 实例
    mla = MultiHeadLatentAttention(hidden_size, down_dim, up_dim, num_heads, rope_head_dim, base, max_len, dropout, bias, device)
    
    # 创建一个随机的 hidden_state
    hidden_state = torch.randn(batch_size, seq_len, hidden_size, device=device)
    
    # 创建一个 attention mask (可选)
    attention_mask = torch.ones(batch_size, seq_len, device=device)
    attention_mask[:, 5:] = 0 # 屏蔽掉每个 batch 中 seq_len 的后 5 个位置
    
    # 通过 MLA 层
    output = mla(hidden_state, attention_mask)
    
    # 打印输出形状
    print("输出形状:", output.shape) # 应输出 torch.Size([2, 10, 256])

输出形状: torch.Size([2, 10, 256])
